# GroupConv Tabanlı Tam Model: **GroupNet** (Kod + Tek Tek İnceleme)

Bu notebook’ta **Group Convolution** kullanan *tam bir CNN modeli* yazacağız ve modeli **parça parça** inceleyeceğiz.

## Bu notebook’ta ne var?
1. Group Conv’un pratikte model içinde **nerede** ve **neden** kullanıldığı
2. GroupNet’in mimarisi (Stem → Stage’ler → Head)
3. Her modülün:
   - Girdi/çıktı şekli (shape)
   - Parametre etkisi
   - Neden o tasarımın seçildiği
4. Kodun tamamı ve çalıştırma örneği

## Önemli mesaj
GroupConv **hesabı düşürür** ama kanal karıştırmayı azaltır.
Bu yüzden modelde bilinçli şekilde **1×1 conv** ile kanal karışımı (channel mixing) yaptıracağız.


---
## 0) Kurulum

Bu notebook yalnızca **PyTorch** ister.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)
print('torch:', torch.__version__)


---
## 1) Model Tasarım Kararı: Neden “Bottleneck + GroupConv”?

GroupConv’u tek başına koymak yerine genelde şu şablon kullanılır:

```
1×1 reduce  →  3×3 group conv  →  1×1 expand
```

Bu şablonun nedeni:

1) **1×1 reduce**: Kanalları düşürür → 3×3’ün maliyeti düşer.
2) **3×3 group conv**: Spatial (uzamsal) işleme ucuzlar.
3) **1×1 expand**: Kanalları geri büyütür ve mixing sağlar.

Bu, ResNet/ResNeXt tarzı mimarilerin temel mantığıdır.


---
## 2) Yardımcı: Parametre Sayımı

Modeli incelerken **parametre** sayısı kritik.
Bu fonksiyon, trainable parametreleri sayar.


In [ ]:
def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)


---
## 3) Temel Blok 1: `ConvBNAct`

Modelin birçok yerinde tekrar eden desen:

- `Conv2d`
- `BatchNorm2d`
- Aktivasyon (ReLU)

Bu yüzden bunu küçük bir modül olarak yazıyoruz.

### Neden BN?
- Aktivasyon dağılımını stabilize eder.
- Eğitimde daha hızlı/sağlam yakınsama sağlar.

### Neden ReLU `inplace=True`?
- Bellek açısından daha verimli.
- (Aşırı edge-case hariç) standart pratik.


In [ ]:
class ConvBNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, groups=1, act=True):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, kernel_size=k, stride=s, padding=p,
                              groups=groups, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.ReLU(inplace=True) if act else nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))


---
## 4) Temel Blok 2: **Grouped Bottleneck (Residual)**

Bu blok GroupConv’un model içindeki “ana çalışma alanı”.

### Blok akışı
1) **Reduce (1×1)**: `C_in → C_mid`
2) **GroupConv (3×3)**: `C_mid → C_mid` (groups=G)
3) **Expand (1×1)**: `C_mid → C_out`
4) **Skip/Residual**: Çıkış + giriş

### Neden residual?
- Derin ağlarda gradient akışını korur.
- Daha stabil ve güçlü temsil.

### Neden `C_mid`?
- `C_mid` küçüldükçe 3×3 maliyeti düşer.
- Çok küçülürse kapasite düşer.
Bu yüzden bir oran (`bottleneck_ratio`) kullanırız.


In [ ]:
class GroupedBottleneck(nn.Module):
    def __init__(self, cin, cout, groups=4, stride=1, bottleneck_ratio=0.5):
        super().__init__()
        # Orta kanal sayısı (bottleneck)
        c_mid = max(1, int(round(cout * bottleneck_ratio)))

        # Reduce: 1x1 (mixing + kanal düşürme)
        self.reduce = ConvBNAct(cin, c_mid, k=1, s=1, p=0)

        # GroupConv: 3x3 (spatial) — burada group conv devreye girer
        # Not: c_mid % groups == 0 olmalı.
        if c_mid % groups != 0:
            # Eğitim notebook’u olduğu için otomatik düzeltme: groups'u küçült
            # (Gerçek projede bu durumda mimariyi yeniden tasarlarsın.)
            for g in range(groups, 0, -1):
                if c_mid % g == 0:
                    groups = g
                    break
        self.groups = groups

        self.gconv = ConvBNAct(c_mid, c_mid, k=3, s=stride, p=1, groups=groups)

        # Expand: 1x1 (kanalı geri büyütme)
        # Son katmanda genelde aktivasyon BLOK DIŞINDA yapılır (ResNet pratiği)
        self.expand_conv = nn.Conv2d(c_mid, cout, kernel_size=1, bias=False)
        self.expand_bn = nn.BatchNorm2d(cout)

        # Skip: shape eşleme (stride veya kanal değişirse)
        if stride != 1 or cin != cout:
            self.skip = nn.Sequential(
                nn.Conv2d(cin, cout, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(cout),
            )
        else:
            self.skip = nn.Identity()

        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = self.skip(x)
        out = self.reduce(x)
        out = self.gconv(out)
        out = self.expand_bn(self.expand_conv(out))
        out = out + identity
        return self.act(out)


### 4.1) Blok Testi (Shape + groups doğrulama)

Aşağıda tek bir blok çalıştırıp:
- Çıkış shape
- Otomatik ayarlanmış `groups`
göreceğiz.


In [ ]:
blk = GroupedBottleneck(cin=64, cout=128, groups=8, stride=2, bottleneck_ratio=0.5)
x = torch.randn(2, 64, 32, 32)
y = blk(x)
print('Used groups:', blk.groups)
print('x:', x.shape, 'y:', y.shape)


---
## 5) Stage Tasarımı

Bir CNN backbone genelde "stage"’lerden oluşur.
Her stage’de:
- İlk blok genelde `stride=2` ile downsample yapar
- Sonraki bloklar `stride=1` ile feature işlemeye devam eder

Bu mantığı bir `make_stage` fonksiyonuyla kuracağız.


In [ ]:
def make_stage(cin, cout, num_blocks, groups, first_stride=2, bottleneck_ratio=0.5):
    blocks = []
    # İlk blok downsample
    blocks.append(GroupedBottleneck(cin, cout, groups=groups, stride=first_stride, bottleneck_ratio=bottleneck_ratio))
    # Devam blokları
    for _ in range(num_blocks - 1):
        blocks.append(GroupedBottleneck(cout, cout, groups=groups, stride=1, bottleneck_ratio=bottleneck_ratio))
    return nn.Sequential(*blocks)


---
## 6) Tam Model: **GroupNet**

### Mimari
1) **Stem**: girişten ilk feature çıkarma
2) **Stage1**: 64 → 128
3) **Stage2**: 128 → 256
4) **Stage3**: 256 → 512
5) **Head**: GlobalAvgPool + Linear

### Tasarım notları
- Stem’i normal conv bırakıyoruz. Çünkü çok erken aşamada "tam mixing" işe yarar.
- Stage’lerde group conv kullanarak maliyeti düşürüyoruz.
- En sonda global pooling ile feature vektörüne indirip sınıflandırıyoruz.


In [ ]:
class GroupNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, groups=8,
                 stage_blocks=(2, 2, 2), bottleneck_ratio=0.5):
        super().__init__()
        self.groups = groups
        self.stage_blocks = stage_blocks

        # Stem
        self.stem = nn.Sequential(
            ConvBNAct(in_channels, 64, k=3, s=1, p=1, groups=1),
            ConvBNAct(64, 64, k=3, s=1, p=1, groups=1),
        )

        # Stages
        self.stage1 = make_stage(64, 128, num_blocks=stage_blocks[0], groups=groups, first_stride=2, bottleneck_ratio=bottleneck_ratio)
        self.stage2 = make_stage(128, 256, num_blocks=stage_blocks[1], groups=groups, first_stride=2, bottleneck_ratio=bottleneck_ratio)
        self.stage3 = make_stage(256, 512, num_blocks=stage_blocks[2], groups=groups, first_stride=2, bottleneck_ratio=bottleneck_ratio)

        # Head
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x, verbose=False):
        if verbose: print('Input:', x.shape)
        x = self.stem(x)
        if verbose: print('After stem:', x.shape)

        x = self.stage1(x)
        if verbose: print('After stage1:', x.shape)

        x = self.stage2(x)
        if verbose: print('After stage2:', x.shape)

        x = self.stage3(x)
        if verbose: print('After stage3:', x.shape)

        x = self.pool(x)
        if verbose: print('After GAP:', x.shape)

        x = torch.flatten(x, 1)
        if verbose: print('Flatten:', x.shape)

        logits = self.fc(x)
        if verbose: print('Logits:', logits.shape)
        return logits


---
## 7) Modeli Çalıştırma + Shape İzleme

Burada forward içinde `verbose=True` vererek **tüm shape akışını** göreceğiz.
Bu sana stage mantığını tam oturtur.


In [ ]:
model = GroupNet(in_channels=3, num_classes=10, groups=8, stage_blocks=(2,2,2), bottleneck_ratio=0.5)
print('Trainable params:', f'{count_params(model):,}')

x = torch.randn(2, 3, 128, 128)
y = model(x, verbose=True)
print('\nFinal output:', y.shape)


---
## 8) Kod İnceleme: `groups` Seçimi Nasıl Yapılmalı?

### Pratik kural
- `groups` arttıkça maliyet düşer
- Ama kanal karışımı azalır

### Yaygın seçimler
- ResNeXt: `groups=32` gibi yüksek cardinality
- Mobil: depthwise (groups=in_channels)

### Bu notebook’ta neden 8?
- Öğrenme amaçlı iyi bir denge.
- Çok agresif değil.

### Kritik teknik şart
GroupConv için `C_mid % groups == 0` olmalı.
Bu yüzden `GroupedBottleneck` içinde eğitim amaçlı otomatik küçültme yaptık.
Gerçek projede bu otomatik “yamayı” koymak yerine kanal planını düzgün kurmak gerekir.


---
## 9) (Opsiyonel) Basit FLOP Sezgisi

Tam FLOP hesabı detaylıdır; ama sezgi şu:

- Normal 3×3: `H*W*C_in*C_out*9`
- Group 3×3:  `H*W*C_in*C_out*9 / G`

Yani `G` arttıkça 3×3 maliyeti yaklaşık `G` kat düşer.

Bu yüzden group conv’u çoğu zaman **en pahalı 3×3’ün yerine** koyarız.
